In [130]:
import re
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

_ = nltk.download("stopwords", quiet=True)
_ = nltk.download("punkt", quiet=True)

In [131]:
# Create the data/02_postprocessing direction if it does not exist
if not os.path.exists("data/02_postprocessing"):
    os.makedirs("data/02_postprocessing")
# Create the data/02_postprocessing/sample directory if it does not exist
if not os.path.exists("data/02_postprocessing/sample"):
    os.makedirs("data/02_postprocessing/sample")

In [132]:
def silent_remove(filename):
    try:
        os.remove(filename)
    except OSError:
        pass

This notebook is designed so that it can be run on the sample data or the full data just by switching one variable.

In [133]:
# Set this to True to run the script on the sample data
# Set this to False to run the script on the full data (takes much longer)
SAMPLE = False

## Ratings

This section performs rating normalization (technically standardization) to all ratings

In [134]:
# Load ratings and users data
file_name = f"data/01_cleaned/{'sample/' if SAMPLE else ''}ratings.csv"
ratings = pd.read_csv(file_name, header=0)
file_name = f"data/01_cleaned/{'sample/' if SAMPLE else ''}users.csv"
users = pd.read_csv(file_name, header=0)

In [135]:
# Index on the "user_id" and "rating_id" columns
ratings = ratings.set_index(["user_id", "rating_id"])
# Index on the "user_id" column for the users data
users = users.set_index("user_id")

In [136]:
# Find the number of ratings for each user_id in the ratings data frame
user_ratings = ratings.groupby("user_id").agg(
    num_ratings = pd.NamedAgg(column="stars", aggfunc="count"),
    mean_rating = pd.NamedAgg(column="stars", aggfunc="mean"),
    std_rating = pd.NamedAgg(column="stars", aggfunc="std"),
)
user_ratings.head()

,num_ratings,mean_rating,std_rating
user_id,,,
--1orhUoGFSdHXsoxqQc8g,1,5.0,NaN
--4ZhTMV2fIlGhcUk8S5rQ,1,4.0,NaN
--8EvC9O7Ycim8vCP4Cj0w,1,3.0,NaN
--8IGwUi6ta3OcJjhAhXgg,1,5.0,NaN
--Hno7jSp4AeOC1vq2Ohew,1,4.0,NaN


In [137]:
# Add a "num_ratings" to the users data frame by joining the users and user_ratings data frames
users = users.join(user_ratings, on="user_id", how="left")
# Set null values to 0
users["num_ratings"].fillna(0, inplace=True)
users.head()

,average_stars,friends,name,review_count,num_ratings,mean_rating,std_rating
user_id,,,,,,,
j14WgRoU_-2ZE1aw1dXrJg,3.74,"['ueRPE0CX75ePGMqOFVj6IQ', '52oH4DrRvzzl8wh5UX...",Daniel,4333,1.0,3.0,NaN
N4Y2GiUxnQOvUUHtFyayKg,3.77,"['nLxx0ZC9ni7ifnjsXLonSQ', 'ium_dLWZUn7N-FUwXT...",Derek,90,0.0,NaN,NaN
SSp_Idg6Ge-Lb6T99kgKSA,5.00,"['QTkOY0n3HPcNy218tawwDg', 'sOJ3jcYn5X_VYVGHDa...",Michael,5,0.0,NaN,NaN
5BpJEC8YVi80ecWT5CtlfQ,3.62,"['nqyyLmjNdS22H0yKSu6-YQ', 'eVUmxx1kN-Acni0t7g...",Jenny,2739,1.0,4.0,NaN
RFTVBFlgM0cqvl1daywfjQ,4.08,"['3Vd_ATdvvuVVgn_YCpz8fw', 'yFLNHDlCriteboCVKj...",Kara,465,1.0,4.0,NaN


In [138]:
# Add a "above_cutoff" column to the users data frame
# This column is True if the user has more than "cutoff" ratings in the ratings data frame
cutoff = 5 # TODO justify this
users["above_cutoff"] = users["num_ratings"] > cutoff
users.head()

,average_stars,friends,name,review_count,num_ratings,mean_rating,std_rating,above_cutoff
user_id,,,,,,,,
j14WgRoU_-2ZE1aw1dXrJg,3.74,"['ueRPE0CX75ePGMqOFVj6IQ', '52oH4DrRvzzl8wh5UX...",Daniel,4333,1.0,3.0,NaN,False
N4Y2GiUxnQOvUUHtFyayKg,3.77,"['nLxx0ZC9ni7ifnjsXLonSQ', 'ium_dLWZUn7N-FUwXT...",Derek,90,0.0,NaN,NaN,False
SSp_Idg6Ge-Lb6T99kgKSA,5.00,"['QTkOY0n3HPcNy218tawwDg', 'sOJ3jcYn5X_VYVGHDa...",Michael,5,0.0,NaN,NaN,False
5BpJEC8YVi80ecWT5CtlfQ,3.62,"['nqyyLmjNdS22H0yKSu6-YQ', 'eVUmxx1kN-Acni0t7g...",Jenny,2739,1.0,4.0,NaN,False
RFTVBFlgM0cqvl1daywfjQ,4.08,"['3Vd_ATdvvuVVgn_YCpz8fw', 'yFLNHDlCriteboCVKj...",Kara,465,1.0,4.0,NaN,False


In [139]:
ratings["std_rating"] = np.NaN
cutoff_users = users[users["above_cutoff"]]
ratings.loc[cutoff_users.index, "std_rating"] = (
    ratings.loc[cutoff_users.index, "stars"] - cutoff_users["mean_rating"]
) / cutoff_users["std_rating"]

In [140]:
ratings.loc[cutoff_users.index].head(20)

business_id  stars  \
user_id                rating_id                                               
VL12EhEdT4OWqGq0nIqkzw qtFnuEL_58MvyVIpEXywPQ  W0vdz23JQtVQX5vJkiCj3g    3.0   
                       l2UFtjSdjWWjVcyw2IcFVQ  0M5mxwnOJJ_Xargxe_Xy3A    4.0   
                       0zECs5P_Wy0eGEmqTyfY-A  fCDMLD21ypv1XZ_EyafCkQ    3.0   
                       BKl0GYlMkW0nudr4pT9zjg  hXvf5a798WvQtFn0Zbh2yw    3.0   
                       dIh0ZXOJ8MigHTC-7l4wjg  tTNGEXBBOg5fLAfxLKBFPw    5.0   
                       VKicGWFFdhjPOxMk1TXmBw  v4jzYZvLqCxPvJLmYzMibg    4.0   
                       x8LMA-wIT0NU2L9CvoZdWw  t7ne_sGR3weuuzaPzxk6kg    4.0   
                       WFIOlBQaufQPTJHo6WsfqA  o8r18rqcRBtmv70VGK5H-A    4.0   
                       WlBxCQPNFl7z-gUgfkN3mg  s_x7XAO2CPMIwWBvzDaNHg    3.0   
                       H_yPLN2J1vGThlE2Xorqjw  yVqIuwyp7WObcTah89gqzg    5.0   
                       KNqHM-wdKGNQBZbnsJmygA  5aOBHxCTNoJ6fhF6KFHSWQ    5.0   
                       ZUYcmj-8_I_4cmK-VC0lkg  -fi_JWvv5mTFXRxDkMg_Tw    5.0   
                       QQFJ-WpGzcZbBN0lPiDg9Q  GH8Pxb6n7CDZehbydWyDcg    1.0   
                       YodJ42c6O-mm7o1jjnTocA  rIdmwXz6QG9qHQwFD9vFSA    5.0   
                       VP99T1HU-XWaTY4nktlsTA  xLuEfxyJ8BnDtI9pG-mGAw    4.0   
                       bPVnC9TVirjOkZlybsFzhw  UihsiUzIWcHmLe4PY0dbSg    1.0   
                       yt_ZdPvHji8Ap6UI7yA0-w  C8KFRgrMtc_naR6I7yWytg    4.0   
                       11QMvCFAzgEkJpuFhGy3Rw  HqgMCVe-jvlP0kT93M_5Sg    5.0   
                       YMyGEAWpWabaQZ0CuuefDQ  xPLsyOpG9kpoXq2j394_Jg    4.0   

                                               std_rating  
user_id                rating_id                           
VL12EhEdT4OWqGq0nIqkzw qtFnuEL_58MvyVIpEXywPQ   -0.642726  
                       l2UFtjSdjWWjVcyw2IcFVQ    0.171394  
                       0zECs5P_Wy0eGEmqTyfY-A   -0.642726  
                       BKl0GYlMkW0nudr4pT9zjg   -0.642726  
                       dIh0ZXOJ8MigHTC-7l4wjg    0.985513  
                       VKicGWFFdhjPOxMk1TXmBw    0.171394  
                       x8LMA-wIT0NU2L9CvoZdWw    0.171394  
                       WFIOlBQaufQPTJHo6WsfqA    0.171394  
                       WlBxCQPNFl7z-gUgfkN3mg   -0.642726  
                       H_yPLN2J1vGThlE2Xorqjw    0.985513  
                       KNqHM-wdKGNQBZbnsJmygA    0.985513  
                       ZUYcmj-8_I_4cmK-VC0lkg    0.985513  
                       QQFJ-WpGzcZbBN0lPiDg9Q   -2.270965  
                       YodJ42c6O-mm7o1jjnTocA    0.985513  
                       VP99T1HU-XWaTY4nktlsTA    0.171394  
                       bPVnC9TVirjOkZlybsFzhw   -2.270965  
                       yt_ZdPvHji8Ap6UI7yA0-w    0.171394  
                       11QMvCFAzgEkJpuFhGy3Rw    0.985513  
                       YMyGEAWpWabaQZ0CuuefDQ    0.171394

In [141]:
other_users = users[~users["above_cutoff"]]
other_mean = ratings.loc[other_users.index, "stars"].mean()
other_std = ratings.loc[other_users.index, "stars"].std()
ratings.loc[other_users.index, "std_rating"] = (
    ratings.loc[other_users.index, "stars"] - other_mean
) / other_std

KeyError: "Index(['N4Y2GiUxnQOvUUHtFyayKg', 'SSp_Idg6Ge-Lb6T99kgKSA',\n       'T1rzNCZw3RdsWE_GnW8FiQ', 'rdysF0ymVkMX82Bu_8x3nA',\n       'WGDlDc6K3zYRgEZSJlqYCw', 'yLljoL8Uxagzzkg85RvGYQ',\n       'X6ikCQsAC4QGN-RjmvW2vA', 'ya3mbvTaymecfrTc08SxhQ',\n       '97YdD8tkMkUSvGUw6_u4GQ', 'hZ_tIZhMhIYd126yoxssvQ',\n       ...\n       'fqWoGghVaEvJEubXU8Ajag', 'y2eB7I_nxWSicimcl-sVQw',\n       'H9QOIaiezjN0lGmU3yLcmA', 'DkbOmgYyHYGVn9ZaZNyXzA',\n       'corDZZAahPfMDs3XRe1Vpw', '2Oyrn6iKeAtIpXlUm-6qmw',\n       'lh1iUwyT_ZlMW0M0Bv2XdQ', 'brKLO1zQ248h3a59B7bATQ',\n       '2XkJjg63m17lotYVe7VL0w', 'hd1sVvOZLoTxIFzl3JiaSw'],\n      dtype='object', name='user_id', length=19295) not in index"

In [142]:
ratings.loc[other_users.index].head(20)

KeyError: "Index(['N4Y2GiUxnQOvUUHtFyayKg', 'SSp_Idg6Ge-Lb6T99kgKSA',\n       'T1rzNCZw3RdsWE_GnW8FiQ', 'rdysF0ymVkMX82Bu_8x3nA',\n       'WGDlDc6K3zYRgEZSJlqYCw', 'yLljoL8Uxagzzkg85RvGYQ',\n       'X6ikCQsAC4QGN-RjmvW2vA', 'ya3mbvTaymecfrTc08SxhQ',\n       '97YdD8tkMkUSvGUw6_u4GQ', 'hZ_tIZhMhIYd126yoxssvQ',\n       ...\n       'fqWoGghVaEvJEubXU8Ajag', 'y2eB7I_nxWSicimcl-sVQw',\n       'H9QOIaiezjN0lGmU3yLcmA', 'DkbOmgYyHYGVn9ZaZNyXzA',\n       'corDZZAahPfMDs3XRe1Vpw', '2Oyrn6iKeAtIpXlUm-6qmw',\n       'lh1iUwyT_ZlMW0M0Bv2XdQ', 'brKLO1zQ248h3a59B7bATQ',\n       '2XkJjg63m17lotYVe7VL0w', 'hd1sVvOZLoTxIFzl3JiaSw'],\n      dtype='object', name='user_id', length=19295) not in index"

In [143]:
min_std = ratings["std_rating"].min()
max_std = ratings["std_rating"].max()
ratings["norm_std_rating"] = (((ratings["std_rating"] - min_std) / (max_std - min_std)) * 4) + 1
ratings.head(20)

,,business_id,stars,std_rating,norm_std_rating
user_id,rating_id,,,,
l_izt6hn7oTxalU06mSyqg,ra9fNjYUumKp_iaqv-jjDg,jTI5Xjk27An8ceJ6VwpXiQ,4.0,NaN,NaN
cARxOd_5yKCgsCbUZ5ED4Q,UqZyX64mgw1vJ1ffnxODjA,EtKSTHV5Qx_Q7Aur9o4kQQ,4.0,NaN,NaN
Wshkpj8aHWuzIeca1QxWTA,OiiUQHpUPSIw592zKebnTg,R-HCwu9UbasUudG1yTM1Ow,1.0,NaN,NaN
ZDUCfIgyPVG7HhFDWhK7kQ,zoKFjcYPeQm7lT9TPqCIuQ,EneM1oBFxncaR1nQCD3gYA,5.0,NaN,NaN
J2sFnwMsDGREmCFkuI-B3Q,OsFuXoyrQwmIwj6Yz7QK8w,dLMA55z9wdDkUIO-rnhOmw,1.0,NaN,NaN
qfm3p0svZIueCPof6Lgjkw,dwgJ7B_r3iuIqX0VpCoRTA,vfBzDd-ceWnCrKqHdZwMSg,5.0,NaN,NaN
0EgbVtQfdNQq3Tapxwl_sw,rnyCUpFeNzS7J0ojgbDKdA,VCjvaX_78oEKG3I9oTJEag,4.0,NaN,NaN
Mj8mM0pGHEsTLn5uuU9mdw,0oN6AFhkj2IOf9gGpfZtxA,FHNIvNgh3fS7VZQq2Y3dsA,4.0,NaN,NaN
YHzVud8gFNn1MLAKsek_Iw,QyvqHQzpNS1Zdiy88BIb3w,dfKF-oAUf3yjnD0WBYDyZQ,5.0,NaN,NaN


In [144]:
reindexed_ratings = ratings.reset_index()
reindexed_ratings.head()

,user_id,rating_id,business_id,stars,std_rating,norm_std_rating
0,l_izt6hn7oTxalU06mSyqg,ra9fNjYUumKp_iaqv-jjDg,jTI5Xjk27An8ceJ6VwpXiQ,4.0,NaN,NaN
1,cARxOd_5yKCgsCbUZ5ED4Q,UqZyX64mgw1vJ1ffnxODjA,EtKSTHV5Qx_Q7Aur9o4kQQ,4.0,NaN,NaN
2,Wshkpj8aHWuzIeca1QxWTA,OiiUQHpUPSIw592zKebnTg,R-HCwu9UbasUudG1yTM1Ow,1.0,NaN,NaN
3,ZDUCfIgyPVG7HhFDWhK7kQ,zoKFjcYPeQm7lT9TPqCIuQ,EneM1oBFxncaR1nQCD3gYA,5.0,NaN,NaN
4,J2sFnwMsDGREmCFkuI-B3Q,OsFuXoyrQwmIwj6Yz7QK8w,dLMA55z9wdDkUIO-rnhOmw,1.0,NaN,NaN


In [145]:
file_name = f"data/02_postprocessing/{'sample/' if SAMPLE else ''}ratings.csv"
with open(file_name, "w") as f:
    reindexed_ratings.to_csv(f, index=False, header=True)

## Businesses

### Categories >> Type, Keywords

In [146]:
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}businesses.csv") as f:
    businesses = pd.read_csv(f, header=0)

In [147]:
# Define category grouping based on similarity
category_groups = {
    'Restaurants': ['Restaurants','Food', 'Breakfast & Brunch', 'Fast Food', 'Burgers', 'Pizza', 'Sandwiches', 'American (Traditional)', 'American (New)', 'Mexican', 'Chicken Wings', 'Salad', 'Chinese', 'Cafes', 'Sushi Bars', 'Barbeque', 'Southern', 'Japanese', 'Steakhouses', 'Juice Bars & Smoothies', 'Asian Fusion', 'Diners', 'Tex-Mex', 'Thai', 'Mediterranean', 'Indian', 'Vietnamese', 'Cajun/Creole', 'Latin American', 'Cuban', 'Puerto Rican', 'Caribbean', 'Japanese', 'Spanish', 'Korean', 'French', 'Halal', 'Mongolian', 'Canadian (New)', 'Filipino', 'Greek', 'Brazilian', 'Argentine', 'Hawaiian', 'Afghan', 'Indonesian', 'German', 'Ramen', 'Poke', 'Hot Pot', 'German', 'Pakistani', 'Uzbek', 'Persian/Iranian', 'Russian', 'Burmese', 'New Mexican Cuisine'],
    'Shopping': ['Shopping', 'Fashion', 'Specialty Food', 'Bakeries', 'Grocery', 'Flowers & Gifts', 'Furniture Stores', 'Jewelry', 'Shoe Stores', 'Accessories', 'Vintage & Consignment', 'Sporting Goods', 'Beer, Wine & Spirits', 'Department Stores', 'Bookstores', 'Electronics', 'Drugstores', 'Music & DVDs', 'Toy Stores', 'Tattoo', 'Eyewear & Opticians', 'Optometrists', 'Comic Books', 'Antiques', 'Gift Shops', 'Mobile Phones', 'Discount Store', 'Hardware Stores', 'Pet Stores', 'Appliances & Repair', 'Appliances', 'Vape Shops', 'Outlet Stores', 'Home & Garden', 'Building Supplies', 'Art Supplies', 'Thrift Stores', 'Hobby Shops', 'Musical Instruments & Teachers', 'Mattresses', 'Bike Repair/Maintenance', 'Books, Mags, Music & Video', 'Computers', 'Mobile Phone Accessories', 'Cosmetics & Beauty Supply', 'Eyelash Service', 'Hair Extensions', 'Hair Stylists', 'Waxing', 'Cosmetic Surgeons', 'Makeup Artists', 'Hair Loss Centers', 'Medical Spas', 'Nail Technicians', 'Hair Salons', 'Nail Salons', 'Massage', 'Day Spas', 'Barbers', 'Massage Therapy', 'Tattoo Removal', 'Piercing', 'Laser Hair Removal', 'Eyebrow Services', 'Permanent Makeup', 'Acne Treatment', 'Weight Loss Centers', 'Medical Supplies', 'Optometrists', 'Health Markets', 'Nutritionists', 'Physical Therapy', 'Dermatologists', 'Chiropractors', 'Hospitals', 'Acupuncture', 'Cryotherapy', 'Medical Centers', 'Sports Medicine', 'Alternative Medicine', 'Prenatal/Perinatal Care', 'Ophthalmologists', 'Internal Medicine', 'Allergists', 'Audiologist', 'Ear Nose & Throat', 'Allergists'],
    'Home Services': ['Home Services', 'Local Services', 'Real Estate', 'Contractors', 'Apartments', 'Home Decor', 'Movers', 'Mattresses', 'Appliances & Repair', 'Appliances', 'Home & Garden', 'Building Supplies', 'Security Systems', 'Waterproofing', 'Insulation Installation', 'Pest Control', 'Heating & Air Conditioning/HVAC', 'Plumbing', 'Carpet Cleaning', 'Air Duct Cleaning', 'Home Inspectors', 'Electricians', 'Solar Installation', 'Solar Panel Cleaning', 'Windows Installation', 'Glass & Mirrors', 'Window Washing', 'Pressure Washers', 'Roofing', 'Gutter Services', 'Siding', 'Carpenters', 'Masonry/Concrete', 'Fireplace Services', 'Carpeting', 'Home Automation', 'Handyman', 'Painters', 'Landscape Architects', 'Irrigation', 'Fences & Gates', 'Pool & Hot Tub Service', 'Pool Cleaners', 'Tree Services', 'Septic Services', 'Water Purification Services', 'Oil Change Stations', 'Excavation Services', 'Snow Removal'],
    'Beauty & Spas': [ 'Beauty & Spas', 'Hair Salons', 'Nail Salons', 'Hair Removal', 'Skin Care', 'Day Spas', 'Barbers', 'Massage', 'Waxing', 'Massage Therapy', 'Tattoo Removal', 'Piercing', 'Laser Hair Removal', 'Eyebrow Services', 'Permanent Makeup', 'Acne Treatment', 'Weight Loss Centers', 'Health Markets', 'Nutritionists', 'Physical Therapy', 'Dermatologists', 'Chiropractors', 'Medical Spas', 'Cosmetic Surgeons', 'Makeup Artists', 'Hair Loss Centers', 'Medical Supplies', 'Optometrists', 'Alternative Medicine', 'Prenatal/Perinatal Care', 'Ophthalmologists', 'Internal Medicine', 'Allergists', 'Audiologist', 'Ear Nose & Throat', 'Allergists'],
    'Nightlife': ['Nightlife', 'Bars', 'Sports Bars', 'Pubs', 'Cocktail Bars', 'Beer Gardens', 'Wine Bars', 'Karaoke', 'Breweries', 'Dive Bars', 'Wine Tasting Room', 'Jazz & Blues', 'Tiki Bars', 'Bartenders', 'Distilleries'],
    'Other': []
}
# Make the lists into sets
for key in category_groups.keys():
    category_groups[key] = set([category.lower() for category in category_groups[key]])

In [148]:
# Make category a list
businesses["categories"] = businesses["categories"].apply(lambda x: x.split(", ") if type(x) == str else [])
# Make category lowercase
businesses["categories"] = businesses["categories"].apply(lambda x: set([category.lower() for category in x]))
# Map according to the category_groups mapping
businesses["categories"] = businesses["categories"].apply(lambda x: [cat_name for cat_name, cat_values in category_groups.items() if len(cat_values.intersection(x)) > 0])
# Pick the top category according to the following order of preference: Restaurants > Shopping > Home Services > Beauty & Spas > Nightlife > Other
def category_sort(category) -> int:
    return ["Restaurants", "Shopping", "Home Services", "Beauty & Spas", "Nightlife", "Other"].index(category)
businesses["categories"] = businesses["categories"].apply(
    lambda x: sorted(list(x), key=category_sort)[0] if len(x) > 0 else "Other")
# Rename category to type
businesses.rename(columns={"categories": "type"}, inplace=True)
businesses.head()

,longitude,name,type,review_count,stars,latitude,business_id
0,-119.711197,"Abby Rappoport, LAC, CMQ",Shopping,7,5.0,34.426679,Pns2l4eNsfO8kk83dixA6A
1,-90.335695,The UPS Store,Home Services,15,3.0,38.551126,mpf3x-BjTdTEA3yCZrAYPw
2,-110.880452,Target,Shopping,22,3.5,32.223236,tUFrWirKiKi_TAnsVWINQQ
3,-75.155564,St Honore Pastries,Restaurants,80,4.0,39.955505,MTSW4McQd7CbVtyjqoe9mw
4,-75.471659,Perkiomen Valley Brewery,Restaurants,13,4.5,40.338183,mWMc6_wTdE0EUBKIGXDVfA


In [149]:
file_name = f"data/02_postprocessing/{'sample/' if SAMPLE else ''}businesses.csv"
with open(file_name, "w") as f:
    businesses.to_csv(f, index=False, header=True)

### Keyword Extraction

In [150]:
# read business JSON file 
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}businesses.csv") as f:
    business_df = pd.read_csv(f, header=0)

In [151]:
stop_words = set(stopwords.words("english"))
custom_stop_words = [",", "&", "-", "(", ")", ".", "'", "!", "?", ":", ";", "[", "]", "/", "I", "'ve", "'s", "n't", "a", "``", "also", "'ll", "$", "'d", " "]
# add numbers to the custom_stop_words list 1 to 100
custom_stop_words.extend([str(i) for i in range(100)])
# combine the stop_words and custom_stop_words lists and make it into a set
all_stop_words = set(stopwords.words("english") + custom_stop_words)

In [152]:
# Extract keywords from each business entry for category and name attributes using dictionary; make lowercase
business_df["categories"] = business_df["categories"].str.lower()
business_df["name"] = business_df["name"].str.lower()

# make a new data frame for keywords from categories and name attributes
# split the categories string into a list of words and remove stop words
# if categories is None don't run
if business_df["categories"].isnull().values.any() == False:
    business_df["categories"] = business_df["categories"].apply(lambda x: [word for word in re.split("\W+", x) if word not in all_stop_words])
business_df["name"] = business_df["name"].apply(lambda x: [word for word in re.split("\W+", x) if word not in all_stop_words])
business_keywords_df = business_df[["business_id", "name", "categories"]].reset_index(drop=True)
# combine the name and categories lists into a keywords list
business_keywords_df["keywords"] = business_keywords_df["name"].astype(str) + business_keywords_df["categories"].astype(str)
# remove the name and categories columns
business_keywords_df = business_keywords_df.drop(columns=["name", "categories"])
# explode the keywords list
business_keywords_addition = business_keywords_df.explode("keywords").reset_index(drop=True)
# rename the column
business_keywords_addition.rename(columns={"keywords": "keyword"}, inplace=True)


In [153]:
# read review JSON file
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}reviews.csv") as f:
    review_df = pd.read_csv(f, header=0)

In [154]:
# combine the stop_words and custom_stop_words lists and make it into a set
all_stop_words = set(stopwords.words("english") + custom_stop_words)

In [155]:
review_df["text"] = review_df["text"].str.lower() # make text lowercase
# Make the text column a list by tokenizing it
review_df["text"] = review_df["text"].apply(lambda x: [word for word in re.split('\W+', x) if word not in all_stop_words])

In [156]:
# Make a new data frame for keywords
review_keywords_df = review_df[["business_id", "text"]].explode("text").rename(columns={"text": "keyword"})

In [157]:
# Create a DataFrame with keyword counts
temp = review_keywords_df.groupby(["business_id", "keyword"]).size().reset_index(name="count")

In [158]:
# For each business id, get the top 10 keywords with the highest count
top_keywords_per_business = (temp.groupby("business_id")
                             .apply(lambda x: x.nlargest(5, "count"))
                             .reset_index(drop=True))

In [159]:
top_keywords_per_business.head(15)

,business_id,keyword,count
0,---kPU91CF4Lq2-WlRu9Lw,nice,3
1,---kPU91CF4Lq2-WlRu9Lw,location,2
2,---kPU91CF4Lq2-WlRu9Lw,back,1
3,---kPU91CF4Lq2-WlRu9Lw,coleslaw,1
4,---kPU91CF4Lq2-WlRu9Lw,coming,1
5,--7PUidqRWpRSpXebiyxTg,chicken,3
6,--7PUidqRWpRSpXebiyxTg,friend,2
7,--7PUidqRWpRSpXebiyxTg,ordered,2
8,--7PUidqRWpRSpXebiyxTg,,1
9,--7PUidqRWpRSpXebiyxTg,acid,1


In [160]:
business_keywords = pd.concat([business_keywords_addition, top_keywords_per_business[["business_id", "keyword"]]], ignore_index=True)
business_keywords.drop_duplicates(inplace=True)
business_keywords.reset_index(drop=True, inplace=True)

# remove the rows with no keywords
business_keywords = business_keywords[business_keywords["keyword"] != ""]

In [161]:
# Store the business keywords in a csv file
file_name = f"data/02_postprocessing/{'sample/' if SAMPLE else ''}businesses_keywords.csv"
with open(file_name, "w") as f:
    business_keywords.to_csv(f, index=False, header=True)